## Quick start demo

### add TileDB.Cloud package, other dependency packages(Newtonsoft.Json, JsonSubtypes, RestSharp, Polly, TileDB.CSharp) will also be added

In [ ]:
#r "nuget:TileDB.Cloud"

### Login

In [ ]:
//string host = "https://api.tiledb.com/v1";
//string api_key = System.Environment.GetEnvironmentVariable("REST_TOKEN_KEY");
//TileDB.Cloud.Client.Login(token:api_key,host:host);
TileDB.Cloud.Client.Login();

### Get User information

In [ ]:
var user_details = TileDB.Cloud.RestUtil.GetUser();
System.Console.WriteLine("{0}", user_details.ToJson());

### Array uri infos

In [ ]:
string original_file = "s3://tiledb-inc-demo-data/files/original/VLDB17_TileDB.pdf";

//TileDB File details
string tiledb_file_name = "VLDB17_TileDB_Example";
string tiledb_file_s3_uri = user_details.DefaultS3Path + "/" + tiledb_file_name;
string uri = "tiledb://" + user_details.Username + "/" + tiledb_file_s3_uri;
System.Console.WriteLine("uri:{0}", uri);

In [ ]:
// Export details
string export_local_path = "exported.pdf";
string export_s3_uri = user_details.DefaultS3Path + "/" + "exported.pdf";
System.Console.WriteLine("export_s3_uri:{0}", export_s3_uri);

### Delete the file array if exists

In [ ]:
var arrayinfo = TileDB.Cloud.RestUtil.GetArrayInfo(uri);
if(arrayinfo != null) {
    TileDB.Cloud.RestUtil.DeleteArray(uri, "application/json");
}
 

### Try to create a file array

In [ ]:
var file_details = TileDB.Cloud.RestUtil.CreateFile(user_details.Username, original_file, tiledb_file_s3_uri, tiledb_file_name);
if(file_details == null) 
{
    System.Console.WriteLine("failed to create file:{0}", tiledb_file_s3_uri);
} 

In [ ]:
arrayinfo = TileDB.Cloud.RestUtil.GetArrayInfo(uri);
if(arrayinfo==null)
{
    System.Console.WriteLine("failed to get array info for uri:{0}", uri);
}
else {
    System.Console.WriteLine("{0}",arrayinfo.ToJson());
}

### Catlog searching

In [ ]:
var arrayList = TileDB.Cloud.RestUtil.ListArrays(user_details.Username);
Console.WriteLine("{0}", arrayList.ToJson());

var listPublicArrays = TileDB.Cloud.RestUtil.ListPublicArrays(user_details.Username);
Console.WriteLine("{0}", listPublicArrays.ToJson());

var listSharedArrays = TileDB.Cloud.RestUtil.ListSharedArrays(user_details.Username);
Console.WriteLine("{0}", listSharedArrays.ToJson());

### Export from tiledb to s3

In [ ]:
var file_exported = TileDB.Cloud.RestUtil.ExportFile(uri, export_s3_uri);
if(file_exported == null) 
{
    System.Console.WriteLine("failed export from {0} to {1}", uri, export_s3_uri );
}


### Export between local file and cloud array

In [ ]:
//default vfs.s3.region is us-east-2
TileDB.Config cfg = new TileDB.Config();
cfg.set("vfs.s3.region", "us-east-2");
TileDB.Context ctx = new TileDB.Context(cfg);

//export file array to local file
TileDB.Cloud.RestUtil.ExportFileArrayToLocalFile("s3://stefan-region-test/image_tdb", "image_exported.tiff", ctx);

//export local file image_exported.tiff
TileDB.Cloud.RestUtil.SaveLocalFileToFileArray("s3://tiledb-bin/image_tdb", "image_exported.tiff","","",ctx);